# TNT Single-Mode: Flow Curve Fitting

> **Handbook:** This notebook demonstrates the TNT Single-Mode model for transient network theory. For complete mathematical derivations and theoretical background, see [TNT Model Family Documentation](../../docs/source/models/tnt/index.rst).

**Estimated Time:** 3-5 minutes (NLSQ fitting ~5s, Bayesian inference demonstration ~30-60s)

## Welcome to Transient Network Theory (TNT)

This notebook is the **gateway** to the TNT (Transient Network Theory) model family — a molecular framework for **associative polymer networks** where reversible crosslinks continuously break and reform under flow.

### What are Transient Networks?

TNT describes materials where polymer chains are connected by **temporary bonds** that dissociate and re-associate on experimental timescales:

- **Telechelic polymers**: Hydrophobic end-groups form micellar clusters
- **Living polymers**: Wormlike micelles with reversible scission (Cates model)
- **Metal-organic networks**: Coordination bonds with thermal activation
- **Hydrogen-bonded assemblies**: Supramolecular structures with directional bonds
- **Loop-bridge systems**: Intra-chain loops vs. inter-chain bridges (shear thickening/thinning)
- **Multi-species networks**: Multiple bond types with different lifetimes
- **Sticky Rouse polymers**: Multi-mode sticker dynamics with hierarchical relaxation

### The TNT Family in RheoJAX (5 Models)

| Model | Key Physics | Best For | Notebooks |
|-------|-------------|----------|-----------|
| **SingleMode** (NB01-06) | Tanaka-Edwards baseline with constant breakage | Learning TNT fundamentals, UCM comparison | This series |
| **Cates** (NB07-12) | Living polymers with reversible scission, $\tau_d = \sqrt{\tau_{rep} \cdot \tau_{break}}$ | Wormlike micelles, Cole-Cole semicircle, shear banding | Flow curve through LAOS |
| **LoopBridge** (NB13-18) | Two-species kinetics (loops carry no stress) | Shear thickening then thinning, non-monotonic viscosity | Flow curve through LAOS |
| **MultiSpecies** (NB19-24) | Multiple bond types with independent evolution | Staged shear thinning, multi-modal relaxation | Flow curve through LAOS |
| **StickyRouse** (NB25-30) | Multi-mode sticker dynamics with Rouse scaling | Hierarchical relaxation, $G'(\omega) \sim \omega^{1/2}$ regime | Flow curve through LAOS |

### Core TNT Principles

All TNT models share the **conformation tensor evolution equation**:

$$
\frac{d\mathbf{S}}{dt} = \mathbf{L} \cdot \mathbf{S} + \mathbf{S} \cdot \mathbf{L}^T + g_0 \mathbf{I} - \beta(\mathbf{S}) \cdot \mathbf{S}
$$

where:
- $\mathbf{S}$: Conformation tensor (2nd moment of chain end-to-end vector distribution)
- $\mathbf{L}$: Velocity gradient tensor (defines flow kinematics)
- $g_0$: Strand creation rate (typically set to $1/\tau_b$ for steady-state balance)
- $\beta(\mathbf{S})$: Breakage rate tensor (constant, stress-dependent, or species-specific)

**Stress-population coupling**: The stress is directly proportional to the conformation:

$$
\boldsymbol{\tau} = G \mathbf{S}
$$

where $G$ is the elastic modulus. The model variants differ primarily in **how bonds break** (the $\beta$ term).

### Single-Mode Model (This Notebook)

The **TNT Single-Mode** model is the Tanaka-Edwards baseline:

- **Constant breakage**: $\beta = 1/\tau_b$ (bond lifetime independent of stress)
- **Result**: Mathematically identical to Upper Convected Maxwell (UCM) — **no shear thinning**
- **Flow curve**: $\sigma = G\tau_b \dot{\gamma} + \eta_s \dot{\gamma}$ (Newtonian)
- **Use case**: Pedagogical baseline to understand conformation tensor mechanics before adding complexity

To get **shear thinning**, use:
- **Bell breakage**: $\beta = (1/\tau_b) \exp(\nu \tau / G)$ (force-dependent dissociation)
- Or switch to Cates/LoopBridge/MultiSpecies/StickyRouse models with inherent nonlinearity

### Six Rheological Protocols (Each Sub-Family)

Each TNT sub-family has **6 protocol notebooks** demonstrating different deformation histories:

1. **FLOW_CURVE** (NB01, 07, 13, 19, 25): Steady-state viscosity vs. shear rate
2. **STARTUP** (NB02, 08, 14, 20, 26): Transient stress overshoot from rest
3. **RELAXATION** (NB03, 09, 15, 21, 27): Exponential decay after cessation of flow
4. **CREEP** (NB04, 10, 16, 22, 28): Delayed strain response to constant stress
5. **SAOS** (NB05, 11, 17, 23, 29): Small-amplitude oscillatory shear (G', G'')
6. **LAOS** (NB06, 12, 18, 24, 30): Large-amplitude nonlinear response

> **TNT Protocols Reference**  
> For the mathematical framework shared by all TNT models, see:  
> [../../docs/source/models/tnt/tnt_protocols.rst](../../docs/source/models/tnt/tnt_protocols.rst)

### Recommended Learning Path

**New to TNT?** Start here:
1. NB01 (this notebook): Flow curve fitting, parameter extraction
2. NB02: Startup shear (stress overshoot)
3. NB05: SAOS (frequency-dependent moduli)
4. NB07: Cates model (shear thinning, living polymers)

**Advanced topics**:
- NB13-18: Loop-bridge dynamics (shear thickening)
- NB19-24: Multi-species (staged shear thinning)
- NB25-30: Sticky Rouse (hierarchical relaxation)

> **TNT Knowledge Extraction Guide**  
> For interpreting fitted parameters and extracting physical insights, see:  
> [../../docs/source/models/tnt/tnt_knowledge_extraction.rst](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)

---

## Learning Objectives (This Notebook)

1. Understand the Tanaka-Edwards transient network theory for associative polymers
2. Fit TNT single-mode model to steady-state flow curve data
3. Extract network parameters: elastic modulus $G$, breakage time $\tau_b$, solvent viscosity $\eta_s$
4. Predict normal stress differences from shear stress
5. Compare constant vs. Bell breakage variants
6. Perform Bayesian inference with NUTS for parameter uncertainty quantification

## Prerequisites

- Basic rheology concepts (shear stress, shear rate, viscosity)
- Familiarity with transient network models (or read NB01-05 first)
- Understanding of Bayesian inference

## 1. Setup

In [ ]:
import os
import sys
import time

# Google Colab support
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTSingleMode

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_ml_ikh_flow_curve,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_single_mode_param_names,
    plot_weissenberg_number_effects,
    plot_variant_comparison,
    compute_tnt_normal_stress,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


## 2. Theory

## Tanaka-Edwards Transient Network Theory

TNT models describe **associative polymer networks** with reversible crosslinks that continuously break and reform. Examples include:
- Telechelic polymers with hydrophobic end-groups
- Metal-organic coordination networks
- Hydrogen-bonded supramolecular assemblies

**Note**: TNT models describe associative polymer networks with reversible crosslinks. **Carbopol (yield-stress fluid) is not the natural target** — we use this data to demonstrate the workflow. For yield-stress fluids, see Fluidity or DMT models.

### Governing Equations

The conformation tensor **S** evolves according to:

$$
\frac{dS}{dt} = \mathbf{L} \cdot \mathbf{S} + \mathbf{S} \cdot \mathbf{L}^T + g_0 \mathbf{I} - \beta(\mathbf{S}) \cdot \mathbf{S}
$$

where:
- $\mathbf{L}$ is the velocity gradient tensor
- $g_0$ is the strand creation rate
- $\beta(\mathbf{S})$ is the breakage rate tensor

### Constant Breakage Variant

For **constant breakage**: $\beta = 1/\tau_b$

At steady state, the flow curve reduces to:

$$
\sigma(\dot{\gamma}) = G \tau_b \dot{\gamma} + \eta_s \dot{\gamma}
$$

This is identical to the **Upper Convected Maxwell (UCM)** model — **no shear thinning**.

### Bell Breakage Variant

For **stress-dependent Bell breakage**: $\beta = (1/\tau_b) \exp(\nu \tau / G)$

This introduces nonlinearity and shear thinning.

### Parameters

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| Elastic modulus | $G$ | Network stiffness | 1-1000 Pa |
| Breakage time | $\tau_b$ | Crosslink lifetime | 0.01-100 s |
| Solvent viscosity | $\eta_s$ | Background viscosity | 0.001-1 Pa·s |
| Bell parameter | $\nu$ | Stress sensitivity (Bell only) | 0.1-2 |

## 3. Load Data

In [ ]:
# Load flow curve data (Carbopol gel)
gamma_dot, stress = load_ml_ikh_flow_curve(instrument="ARES_up")

print(f"Data shape: {len(gamma_dot)} points")
print(f"Shear rate range: {gamma_dot.min():.3e} - {gamma_dot.max():.3e} s⁻¹")
print(f"Stress range: {stress.min():.2f} - {stress.max():.2f} Pa")

In [ ]:
# Plot raw data
fig, ax = plt.subplots(figsize=(8, 5))
ax.loglog(gamma_dot, stress, 'o', label='Data', markersize=8, alpha=0.7)
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s⁻¹)', fontsize=12)
ax.set_ylabel(r'Shear stress $\sigma$ (Pa)', fontsize=12)
ax.set_title('Flow Curve: Carbopol Gel', fontsize=14, fontweight='bold')
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.close("all")

## 4. NLSQ Fitting

In [ ]:
# Create model and fit
model = TNTSingleMode(breakage="constant")

start_time = time.time()
result = model.fit(gamma_dot, stress, test_mode="flow_curve", method="scipy")
fit_time = time.time() - start_time

print(f"\nNLSQ fitting completed in {fit_time:.2f} seconds")
print(f"\nFitted Parameters:")
param_names = get_tnt_single_mode_param_names(breakage="constant")
for name in param_names:
    param = model.parameters.get(name)
    print(f"  {name} = {param.value:.4e}")

In [ ]:
# Compute fit quality metrics
metrics = compute_fit_quality(stress, model.predict(gamma_dot, test_mode="flow_curve"))
r2 = metrics["R2"]
rmse = metrics["RMSE"]
mape = metrics.get("NRMSE", 0.0) * 100  # NRMSE as percentage
print(f"\nFit Quality:")
print(f"  R² = {r2:.6f}")
print(f"  RMSE = {rmse:.4e} Pa")
print(f"  MAPE = {mape:.2f}%")

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e} Pa")
print(f"  Std residual = {np.std(residuals):.4e} Pa")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e} Pa")

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    gamma_dot, stress, model, test_mode="flow_curve",
    param_names=param_names, log_scale=True,
    xlabel=r'Shear rate $\dot{\gamma}$ (s$^{-1}$)',
    ylabel=r'Shear stress $\sigma$ (Pa)',
    title=f'NLSQ Fit (R\u00b2 = {metrics["R2"]:.4f})'
)
plt.close("all")

### 4.2 Physical Analysis

In [ ]:
gamma_dot_fine = np.logspace(np.log10(gamma_dot.min()), np.log10(gamma_dot.max()), 200)
stress_pred = model.predict(gamma_dot_fine, test_mode='flow_curve')
# Extract fitted parameters
G = model.parameters.get('G').value
tau_b = model.parameters.get('tau_b').value
eta_s = model.parameters.get('eta_s').value

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Weissenberg number analysis
ax = axes[0, 0]
Wi = gamma_dot_fine * tau_b
ax.loglog(gamma_dot_fine, Wi, '-', linewidth=2, color='C2')
ax.axhline(y=1, color='k', linestyle='--', alpha=0.5, label='Wi = 1')
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s⁻¹)', fontsize=11)
ax.set_ylabel(r'Weissenberg number $Wi = \dot{\gamma}\tau_b$', fontsize=11)
ax.set_title('Weissenberg Number', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 2. Normal stress prediction
ax = axes[0, 1]
N1 = compute_tnt_normal_stress(gamma_dot_fine, G, tau_b)
ax.loglog(gamma_dot_fine, stress_pred, '-', label=r'$\sigma$ (shear)', linewidth=2)
ax.loglog(gamma_dot_fine, N1, '--', label=r'$N_1$ (normal)', linewidth=2)
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s⁻¹)', fontsize=11)
ax.set_ylabel('Stress (Pa)', fontsize=11)
ax.set_title('Normal Stress Prediction', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 3. Viscosity decomposition
ax = axes[1, 0]
eta_total = stress_pred / gamma_dot_fine
eta_network = G * tau_b * np.ones_like(gamma_dot_fine)
ax.loglog(gamma_dot_fine, eta_total, '-', label=r'$\eta_{total}$', linewidth=2)
ax.axhline(y=G*tau_b, color='C1', linestyle='--', label=r'$\eta_{network} = G\tau_b$', linewidth=2)
ax.axhline(y=eta_s, color='C2', linestyle=':', label=r'$\eta_s$', linewidth=2)
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s⁻¹)', fontsize=11)
ax.set_ylabel('Viscosity (Pa·s)', fontsize=11)
ax.set_title('Viscosity Decomposition', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 4. Variant comparison (constant vs. Bell)
ax = axes[1, 1]
ax.loglog(gamma_dot, stress, 'o', label='Data', markersize=8, alpha=0.7)
ax.loglog(gamma_dot_fine, stress_pred, '-', label='Constant breakage', linewidth=2)
try:
    model_bell = TNTSingleMode(breakage="bell")
    result_bell = model_bell.fit(gamma_dot, stress, test_mode="flow_curve", method="scipy")
    stress_bell = model_bell.predict(gamma_dot_fine, test_mode="flow_curve")
    ax.loglog(gamma_dot_fine, stress_bell, '--', label='Bell breakage', linewidth=2)
except Exception as e:
    print(f"Bell variant comparison skipped: {type(e).__name__}")
ax.set_xlabel(r'Shear rate $\dot{\gamma}$ (s⁻¹)', fontsize=11)
ax.set_ylabel(r'Shear stress $\sigma$ (Pa)', fontsize=11)
ax.set_title('Breakage Variant Comparison', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")

print(f"\nPhysical Interpretation:")
print(f"  Network modulus: G = {G:.2f} Pa")
print(f"  Breakage time: τ_b = {tau_b:.4f} s")
print(f"  Network viscosity: η_network = G·τ_b = {G*tau_b:.2f} Pa·s")
print(f"  Solvent viscosity: η_s = {eta_s:.4e} Pa·s")
print(f"  Viscosity ratio: η_network/η_s = {(G*tau_b)/eta_s:.1f}")

## 5. Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        gamma_dot, stress,
        test_mode='flow_curve',
        
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayesian_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")

### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start

## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        gamma_dot,
        stress,
        model, bayesian_result, test_mode="flow_curve",
        param_names=param_names, log_scale=True,
        xlabel=r'Shear rate $\\dot{\\gamma}$ (s$^{-1}$)',
        ylabel=r'Shear stress $\\sigma$ (Pa)'
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

In [ ]:
# Parameter comparison table
print_parameter_comparison(model, bayesian_result.posterior_samples, param_names)

## 6. Physical Interpretation

### Constant Breakage = UCM Model

For constant breakage rate $\beta = 1/\tau_b$, the TNT model reduces to the Upper Convected Maxwell (UCM) model:

$$
\sigma = G \tau_b \dot{\gamma} + \eta_s \dot{\gamma} = (\eta_{network} + \eta_s) \dot{\gamma}
$$

This is a **Newtonian response** — no shear thinning. The flow curve is linear in log-log space with slope 1.

### Physical Parameters

- **Network modulus G**: Stiffness of the transient network
- **Breakage time τ_b**: Average lifetime of crosslinks
- **Network viscosity η_network = G·τ_b**: Polymer contribution
- **Solvent viscosity η_s**: Background solvent contribution

### Weissenberg Number

The Weissenberg number $Wi = \dot{\gamma} \tau_b$ compares the flow timescale to the relaxation timescale:
- $Wi \ll 1$: Linear viscoelastic regime (network relaxes faster than deformation)
- $Wi \sim 1$: Transition regime
- $Wi \gg 1$: Nonlinear regime (deformation faster than relaxation)

For constant breakage, there is no shear thinning even at high Wi.

### Normal Stress Differences

The TNT model predicts normal stress differences:

$$
N_1 = 2G(\tau_b \dot{\gamma})^2
$$

This is a signature of viscoelasticity — normal stresses arise from polymer stretching.

### Bell Breakage for Shear Thinning

To capture shear thinning, use **Bell breakage**: $\beta = (1/\tau_b) \exp(\nu \tau / G)$

This introduces stress-dependent bond breaking, leading to a decrease in viscosity at high shear rates.

## 7. Save Results

In [ ]:
# Save results
save_tnt_results(model, bayesian_result, "single_mode", "flow_curve", param_names)
print("\nResults saved successfully!")

## Key Takeaways

1. **TNT constant breakage = UCM**: Linear flow curve, no shear thinning
2. **Three parameters**: G (modulus), τ_b (breakage time), η_s (solvent viscosity)
3. **Weissenberg number**: Wi = γ̇·τ_b characterizes flow regime
4. **Normal stresses**: N₁ = 2G(τ_b·γ̇)² from polymer stretching
5. **Bayesian inference**: Quantifies parameter uncertainty with NUTS
6. **Bell variant**: Adds stress-dependent breakage for shear thinning

## Next Steps

- **Notebook 02**: Startup shear with stress overshoot
- **Notebook 03**: Stress relaxation for single-exponential decay
- **Notebook 05**: SAOS for frequency-dependent moduli
- **Notebook 06**: LAOS for nonlinear response

## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models with decision trees and feature comparison
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework and equations shared by all TNT variants
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters and extracting physical insights

### Next Notebooks in SingleMode Series

- **NB02**: Startup shear with stress overshoot
- **NB03**: Stress relaxation (single-exponential decay)
- **NB04**: Creep compliance
- **NB05**: SAOS frequency sweeps
- **NB06**: LAOS nonlinear response

### Advanced TNT Models

- **NB07-12**: Cates model (living polymers, shear banding)
- **NB13-18**: Loop-Bridge model (shear thickening/thinning)
- **NB19-24**: Multi-Species model (staged shear thinning)
- **NB25-30**: Sticky Rouse model (hierarchical relaxation)

### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures